# Cleaning the data

Before I start deleting and modifying stuff, it's good to have a look to the table

In [1]:
import pandas as pd
import src.funciones_toni as tn
import re
import numpy as np

In [2]:
sharkraw = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

In [3]:
sharkraw.shape

(25723, 24)

In [4]:
sharkraw.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [5]:
sharkraw.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

Some column names are wrong, with space after the name, this can lead to confusion and errors later, so I change it right now.

In [6]:
sharkraw.rename(columns = {'Sex ':'Sex'}, inplace = True)
sharkraw.rename(columns = {'Species ':'Species'}, inplace = True)
sharkraw.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [7]:
shark2=sharkraw.drop_duplicates()
shark2.shape

(6312, 24)

With this I can see that lots of rows were empty. From 25.000 original rows now I have a 6.300.

In [8]:
shark2.isnull().sum().sort_values(ascending=False)

Unnamed: 22               6311
Unnamed: 23               6310
Time                      3364
Species                   2848
Age                       2841
Sex                        575
Activity                   554
Location                   550
Fatal (Y/N)                549
Area                       465
Name                       220
Country                     60
Injury                      38
Investigator or Source      27
Type                        14
Year                        12
href formula                11
pdf                         10
href                        10
Case Number.1               10
Case Number.2               10
Date                        10
original order               3
Case Number                  2
dtype: int64

There are some columns with lots of missing information (NaN values), in particular, Unnamed22 and Unnamed23. 

In [9]:
shark2["Species"].value_counts(dropna=False).head(15)

NaN                                                   2848
White shark                                            163
Shark involvement prior to death was not confirmed     105
Invalid                                                102
Shark involvement not confirmed                         88
Tiger shark                                             73
Shark involvement prior to death unconfirmed            68
Bull shark                                              52
4' shark                                                40
6' shark                                                40
1.8 m [6'] shark                                        35
Questionable incident                                   35
Questionable                                            34
1.5 m [5'] shark                                        32
1.2 m [4'] shark                                        27
Name: Species, dtype: int64

This information seems interesting for me, however, the data is very poorly registered. There are lots of long sentences meaning the same.
Regex:
- Sharks defined by names (Capital + shark): ([A-Z][a-z]*\sshark)
- Sharks defined by lenght (in yrds): \d+']\sshark|\d+'\sshark
- Not confirmed: (not confirmed)|(Invalid)|(Questionable)|(unconfirmed)

In [10]:
shark2["Species"]=shark2["Species"].astype(str)
shark2.head()

<ipython-input-10-b380447bc5f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2["Species"]=shark2["Species"].astype(str)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,nan,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,nan,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [11]:
pattern= "[A-Z][a-z]*\sshark|\d+']\sshark|\d+'\sshark"
shark2['Species_sorted'] = shark2['Species'].apply(lambda x: tn.regeshark(x, pattern))

<ipython-input-11-b146e8f0de9f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2['Species_sorted'] = shark2['Species'].apply(lambda x: tn.regeshark(x, pattern))


In [12]:
shark2['Species_sorted'].value_counts(dropna=False).head(20)

NaN                  4391
White shark           436
Tiger shark           237
5' shark              131
Bull shark            130
6' shark              104
4' shark               98
8' shark               51
Nurse shark            49
Wobbegong shark        46
3' shark               44
Mako shark             44
Raggedtooth shark      43
10' shark              41
12' shark              40
7' shark               36
Blacktip shark         34
Lemon shark            32
Zambesi shark          29
Blue shark             29
Name: Species_sorted, dtype: int64

Now the shark species column look sorted and clean.

I'm going to do the same with the Date column, there are 5 columns with information about the date, i.e. "Case Number",	"Date", "Year", "Case Number.1" and "Case Number.2". The year can give some information, so I'm keeping this column, from the other 4 columns I'm only interested in taking the month from the "Date" column, I want to see if there is a correlation between the time of the year and the shark attacks.

In [13]:
pattern="[A-Z][a-z]+"


shark2["Date"]=shark2["Date"].astype(str)

shark2['Month_attack'] = shark2['Date'].apply(lambda x: tn.monthattack(x, pattern))

<ipython-input-13-f5ada1837d87>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2["Date"]=shark2["Date"].astype(str)
<ipython-input-13-f5ada1837d87>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2['Month_attack'] = shark2['Date'].apply(lambda x: tn.monthattack(x, pattern))


In [14]:
shark2['Month_attack'].sample(10)

813     Mar
5101    Jul
6289    Jul
3916    Mar
3598    Dec
2376    Jun
4682    Mar
934     Mar
348     Oct
2381    Jun
Name: Month_attack, dtype: object

Now I have a column with the months where the attacks took place.

The columns "Activity" and "Type" look also interesting, but they already look pretty clean for me, so I'm keeping them as received.

In [15]:
shark2['Activity'].value_counts(dropna=False).head(10)

Surfing         971
Swimming        869
NaN             554
Fishing         431
Spearfishing    333
Bathing         162
Wading          149
Diving          127
Standing         99
Snorkeling       89
Name: Activity, dtype: int64

In [16]:
shark2['Type'].value_counts(dropna=False).head(10)

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
NaN               14
Questionable       2
Boatomg            1
Name: Type, dtype: int64

The column "Time" mixes hours and daytime data, I think it will bring more information if I sort it by morning, afternoon, evening and night.

In [17]:
shark2['Time'].value_counts(dropna=False).head(20)

NaN          3364
Afternoon     187
11h00         128
Morning       121
12h00         109
15h00         108
16h00         101
14h00          97
16h30          74
17h00          73
14h30          73
13h00          72
17h30          70
18h00          69
15h30          66
Night          62
11h30          61
13h30          60
10h00          59
09h00          50
Name: Time, dtype: int64

In [18]:
pattern="^\d{2}|[A-Z][a-z]*"

shark2["Time"]=shark2["Time"].astype(str)
shark2['Time'] = shark2['Time'].apply(lambda x: tn.daytime(x, pattern))

<ipython-input-18-5ce2f662a18c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2["Time"]=shark2["Time"].astype(str)
<ipython-input-18-5ce2f662a18c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2['Time'] = shark2['Time'].apply(lambda x: tn.daytime(x, pattern))


In [19]:
shark2['Time'].value_counts(dropna=False).head()

NaN          3557
Afternoon    1500
Morning       890
Evening       264
Night         101
Name: Time, dtype: int64

Injury column looks clean, there is not much information to get from it, but I'll keep it.

In [20]:
shark2['Fatal (Y/N)'].value_counts(dropna=False).head(50)

N          4293
Y          1388
NaN         549
UNKNOWN      71
 N            7
N             1
y             1
M             1
2017          1
Name: Fatal (Y/N), dtype: int64

In [21]:
pattern='^N|^Y'

shark2['Fatal (Y/N)']=shark2['Fatal (Y/N)'].astype(str)
shark2['Fatal (Y/N)']=shark2['Fatal (Y/N)'].apply(lambda x: tn.fatal(x, pattern))

<ipython-input-21-f34b92fdd683>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2['Fatal (Y/N)']=shark2['Fatal (Y/N)'].astype(str)
<ipython-input-21-f34b92fdd683>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2['Fatal (Y/N)']=shark2['Fatal (Y/N)'].apply(lambda x: tn.fatal(x, pattern))


In [22]:
shark2['Fatal (Y/N)'].value_counts(dropna=False).head()

N      4294
Y      1388
NaN     630
Name: Fatal (Y/N), dtype: int64

There are some values in the Age column with string type, I need it as integers.

In [23]:
shark2['Age'].value_counts(dropna=False).head()

NaN    2841
17      154
18      150
19      142
20      141
Name: Age, dtype: int64

In [24]:
pattern='\d{2}'
    
shark2['Age']=shark2['Age'].astype(str)
shark2['Age']=shark2['Age'].apply(lambda x: tn.age(x, pattern))

<ipython-input-24-2829dd9f346c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2['Age']=shark2['Age'].astype(str)
<ipython-input-24-2829dd9f346c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2['Age']=shark2['Age'].apply(lambda x: tn.age(x, pattern))


In [25]:
shark2['Age']

0        57
1        11
2        48
3         0
4         0
         ..
6307      0
6308      0
6309      0
8702      0
25722     0
Name: Age, Length: 6312, dtype: int64

In the Sex column there are some wrong values, so let's clean it as well

In [26]:
shark2['Sex'].value_counts(dropna=False).head()

M      5094
F       637
NaN     575
M         2
N         2
Name: Sex, dtype: int64

In [27]:
pattern='[M,F]'
    
shark2['Sex']=shark2['Sex'].astype(str)
shark2['Sex']=shark2['Sex'].apply(lambda x: tn.sex(x, pattern))

<ipython-input-27-f58ae79668b3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2['Sex']=shark2['Sex'].astype(str)
<ipython-input-27-f58ae79668b3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark2['Sex']=shark2['Sex'].apply(lambda x: tn.sex(x, pattern))


In [28]:
shark2['Sex'].value_counts(dropna=False).head()

M      5096
F       637
NaN     579
Name: Sex, dtype: int64

In [29]:
shark2.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23', 'Species_sorted', 'Month_attack'],
      dtype='object')

In [30]:
shark2.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Species_sorted,Month_attack
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN,White shark,Jun
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN,NaN,Jun
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN,NaN,Jun
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN,NaN,Jun
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN,Tiger shark,Jun


Now, all the columns I need are clean and sorted, I only have to create a new dataset with the columns I want and import it.

In [31]:
shark_clean=shark2[['Year', 'Month_attack', 'Time', 'Type', 'Activity', 'Country', 'Area', 'Location', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Species_sorted']]

In [32]:
shark_clean=shark_clean.drop_duplicates()


In [33]:
shark_clean

,Year,Month_attack,Time,Type,Activity,Country,Area,Location,Sex,Age,Injury,Fatal (Y/N),Species_sorted
0,2018.0,Jun,Evening,Boating,Paddling,USA,California,"Oceanside, San Diego County",F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark
1,2018.0,Jun,Afternoon,Unprovoked,Standing,USA,Georgia,"St. Simon Island, Glynn County",F,11,Minor injury to left thigh,N,NaN
2,2018.0,Jun,Morning,Invalid,Surfing,USA,Hawaii,"Habush, Oahu",M,48,Injury to left lower leg from surfboard skeg,N,NaN
3,2018.0,Jun,NaN,Unprovoked,Surfing,AUSTRALIA,New South Wales,Arrawarra Headland,M,0,Minor injury to lower leg,N,NaN
4,2018.0,Jun,NaN,Provoked,Free diving,MEXICO,Colima,La Ticla,M,0,Lacerations to leg & hand shark PROVOKED INCIDENT,N,Tiger shark
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6298,0.0,NaN,NaN,Unprovoked,Pearl diving,AUSTRALIA,Western Australia,NaN,M,0,FATAL,Y,NaN
6299,0.0,NaN,NaN,Unprovoked,Swimming,USA,North Carolina,Ocracoke Inlet,M,0,FATAL,Y,NaN
6300,0.0,NaN,NaN,Unprovoked,NaN,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",M,0,FATAL,Y,NaN
6301,0.0,NaN,NaN,Unprovoked,Swimming,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN


In [34]:
shark_clean.to_csv("shark_clean", sep=',')
